In [25]:
from transformers import AutoTokenizer, AutoModel
import torch
from src.dataloader import DataLoader
from utils.utils import flatten_list, write_list_to_file, read_list_from_file, preprocess_text
from bs4 import BeautifulSoup
from src.faiss.flat_idx import FlatIdx
from src.encoder.dragon import Encoder
from src.post_processing import idk
import pandas as pd
from src.eval import Eval
import numpy as np
from fuzzywuzzy import fuzz
import tqdm

In [36]:
!pip install git+https://github.com/naver/splade.git

  Running command git clone --filter=blob:none --quiet https://github.com/naver/splade.git 'C:\Users\J C SINGLA\AppData\Local\Temp\pip-req-build-pj0l1_8s'
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\tokenizers
      copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers
      creating build\lib.win-amd64-cpython-312\tokenizers\models
      copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\models
      creating build\lib.win-amd64-cpython-312\tokenizers\decoders
      copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\decoders
      creating build\lib.win-amd64-cpython-312\tokeniz


  Cloning https://github.com/naver/splade.git to c:\users\j c singla\appdata\local\temp\pip-req-build-pj0l1_8s
  Resolved https://github.com/naver/splade.git to commit 6a1b4bda1cdd5b7117b8387eb831db6d7ce49eeb
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/70.3 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/70.3 kB ? eta -:--:--
     ---------------- --------------------- 30.7/70.3 kB 325.1 kB/s eta 0:00:01
     --------------------------------- ---- 61.4/70.3 kB 469.7 kB/s eta 0:00:01
     -------------------------------------- 70.3/70.3 kB 319.5 kB/s eta 0:00:00
  Using cached omegaconf-2.1.2-py3-none-any.whl.metadata (3.6 kB)
  Using cached antlr4-python3-runtime-4.8.tar.gz (112 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/220.7 kB ? eta -:--:--
    

## Load data

In [5]:
save_path = r"C:\Users\J C SINGLA\Downloads\External - take_home_challenge_(withJSONs)\take_home_challenge_(withJSONs)\data"

In [7]:
all_data_sherpa = read_list_from_file(save_path, "sherpa_paras_and_tables")
filenames_sherpa = read_list_from_file(save_path, "sherpa_paras_and_tables_filenames")
assert (len(all_data_sherpa)==len(filenames_sherpa))
print (len(all_data_sherpa))

502


In [9]:
ground_truth_path = r"C:\Users\J C SINGLA\Downloads\External - take_home_challenge_(withJSONs)\take_home_challenge_(withJSONs)\document_questions.xlsx"
ground_truth = pd.read_excel(ground_truth_path)
ground_truth_text = ground_truth[ground_truth["complexity"].isin(["no raw_text", "table", "text"])].copy()  
test_data = list(ground_truth_text["relevant questions"])
test_labels = list(ground_truth_text["answer"])

In [10]:
len(test_labels)

33

## Load model

In [38]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

model_id = 'naver/splade-cocondenser-ensembledistil'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)